# 10.데이터 집계와 그룹 연산

## 10.1 GroupBy 메카닉

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,0.096256,0.321330
1,a,two,-0.118802,-0.947072
2,b,one,-0.367049,-1.012975
3,b,two,-0.350993,-0.216049
4,a,one,1.371992,-0.252246


In [2]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [3]:
grouped.mean()

key1
a    0.449815
b   -0.359021
Name: data1, dtype: float64

In [4]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one     0.734124
      two    -0.118802
b     one    -0.367049
      two    -0.350993
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,0.734124,-0.118802
b,-0.367049,-0.350993


In [6]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

years = np.array([2005,2005,2006,2005,2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.118802
            2006   -0.367049
Ohio        2005   -0.127368
            2006    1.371992
Name: data1, dtype: float64

길이만 같다면 어떤걸로 묶어도 상관없음

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.449815,-0.292663
b,-0.359021,-0.614512


In [8]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.734124  0.034542
     two  -0.118802 -0.947072
b    one  -0.367049 -1.012975
     two  -0.350993 -0.216049

In [9]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1. 그룹 간 순회하기

In [10]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.096256  0.321330
1    a  two -0.118802 -0.947072
4    a  one  1.371992 -0.252246
b
  key1 key2     data1     data2
2    b  one -0.367049 -1.012975
3    b  two -0.350993 -0.216049


- groupby는 이터레이션을 지원
- 데이터 묶음을 튜플로 반환

In [11]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.096256  0.321330
4    a  one  1.371992 -0.252246
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.118802 -0.947072
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.367049 -1.012975
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.350993 -0.216049


In [12]:
pieces = dict(list(df.groupby('key1')))

pieces['b']

,key1,key2,data1,data2
2,b,one,-0.367049,-1.012975
3,b,two,-0.350993,-0.216049


사전형으로 쉽게 만들 수도 있음

In [13]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

타입에 따라 묶을 수도 있음

In [14]:
grouped = df.groupby(df.dtypes, axis = 1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.096256  0.321330
1 -0.118802 -0.947072
2 -0.367049 -1.012975
3 -0.350993 -0.216049
4  1.371992 -0.252246
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [15]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.034542
     two  -0.947072
b    one  -1.012975
     two  -0.216049

데이터프레임 형식으로 받으려면 대괄호 두개

- [ [ ] ] : DataFrameGroupBy 객체
- \[ \] : SeriesFrameGroupBy 객체

In [16]:
s_grouped = df.groupby(['key1','key2'])['data2']

s_grouped

In [17]:
s_grouped.mean()

key1  key2
a     one     0.034542
      two    -0.947072
b     one    -1.012975
      two    -0.216049
Name: data2, dtype: float64

### 10.1.3 사전과 Series에서 그룹핑하기

In [18]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = ['a','b','c','d','e'],
                      index = ['Joe','Steve','Wes','Jim','Travis'])

people.iloc[2:3, [1,2]] = np.nan #결측치 부여하기
people

,a,b,c,d,e
Joe,0.947468,1.182453,0.377912,1.390904,0.172336
Steve,0.257712,0.052389,-1.596647,-1.296329,-0.441214
Wes,-1.101866,NaN,NaN,0.000032,-1.463199
Jim,2.657548,-0.454724,1.107933,1.734163,-1.956087
Travis,-0.045618,-0.578072,0.553213,0.181805,0.947972


매핑을 통해 그룹핑 가능함

In [19]:
mapping = {'a':'red', 'b':'red','c':'blue','d':'blue','e':'red','f':'orange'}

In [20]:
people.groupby(mapping, axis = 1).sum()

,blue,red
Joe,1.768816,2.302257
Steve,-2.892976,-0.131112
Wes,0.000032,-2.565065
Jim,2.842096,0.246737
Travis,0.735019,0.324282


In [21]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

사전이 아니고 시리즈여도 가능

In [22]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 함수로 그룹핑하기

In [23]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.503150,0.727729,1.485845,3.125099,-3.246949
5,0.257712,0.052389,-1.596647,-1.296329,-0.441214
6,-0.045618,-0.578072,0.553213,0.181805,0.947972


이름의 길이별로 묶으려면 len함수 사용

In [24]:
key_list = ['one','one','one','two','two']

people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -1.101866  1.182453  0.377912  0.000032 -1.463199
  two  2.657548 -0.454724  1.107933  1.734163 -1.956087
5 one  0.257712  0.052389 -1.596647 -1.296329 -0.441214
6 two -0.045618 -0.578072  0.553213  0.181805  0.947972

### 10.1.5 색인 단계로 그룹핑하기

In [25]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                     [1,3,5,1,3]],
                                     names=['cty','tenor'])

hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.789966 -0.169703  0.722574 -0.147854 -0.408002
1     -0.008014  0.404593 -0.720942 -1.435492  2.331700
2     -0.672741  0.169621 -1.710133  0.624304  0.024158
3     -1.798619  0.944344 -1.505019  0.574804 -0.360147

In [26]:
hier_df.groupby(level = 'cty' ,axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


색인 단계는 level 인자로 구분해줌

## 10.2 데이터 집계

In [27]:
df

,key1,key2,data1,data2
0,a,one,0.096256,0.321330
1,a,two,-0.118802,-0.947072
2,b,one,-0.367049,-1.012975
3,b,two,-0.350993,-0.216049
4,a,one,1.371992,-0.252246


In [28]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.116845
b   -0.352598
Name: data1, dtype: float64

In [29]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.490794,1.268402
b,0.016057,0.796926


집계 함수를 만들어 사용할 경우 agg 메서드를 이용해 넘긴다

In [30]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.449815  0.805835 -0.118802 -0.011273  0.096256  0.734124   
b      2.0 -0.359021  0.011354 -0.367049 -0.363035 -0.359021 -0.355007   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.371992   3.0 -0.292663  0.635166 -0.947072 -0.599659 -0.252246   
b    -0.350993   2.0 -0.614512  0.563512 -1.012975 -0.813744 -0.614512   

                          
           75%       max  
key1                      
a     0.034542  0.321330  
b    -0.415281 -0.216049

### 10.2.1 컬럼에 여러 가지 함수 적용하기

In [31]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [32]:
grouped = tips.groupby(['day','smoker'])

grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [33]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [34]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

### 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

In [35]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply:일반적인 분리-적용-병합

상위 5개의 tip_pct 값을 뽑자

In [36]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [38]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

상위 뽑아내는 함수를 만든 뒤 원하는 그룹별로 뽑게 할 수 있다.

In [39]:
tips.groupby(['smoker','day']).apply(top,n=2,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [40]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [41]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [42]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### 10.3.1 그룹 색인 생략하기

In [43]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 변위치 분석과 버킷 분석

In [44]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                      'data2':np.random.randn(1000)})

quartiles = pd.cut(frame.data1,4)

quartiles[:10]

0    (-1.694, -0.245]
1    (-1.694, -0.245]
2    (-1.694, -0.245]
3     (-0.245, 1.204]
4    (-1.694, -0.245]
5     (-0.245, 1.204]
6    (-1.694, -0.245]
7    (-1.694, -0.245]
8    (-1.694, -0.245]
9    (-1.694, -0.245]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.149, -1.694] < (-1.694, -0.245] < (-0.245, 1.204] < (1.204, 2.652]]

In [45]:
def get_stats(group):
    return {'min':group.min(), 'max':group.max(), 'count':group.count(), 'mean':group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.149, -1.694]",-3.067030,1.995843,44.0,-0.284738
"(-1.694, -0.245]",-2.788961,2.903898,359.0,-0.063716
"(-0.245, 1.204]",-2.501937,2.975031,486.0,0.044774
"(1.204, 2.652]",-2.276854,1.865283,111.0,-0.089627


### 10.3.3 예제:그룹에 따른 값으로 결측치 채우기

In [46]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.190854
2         NaN
3    0.035340
4         NaN
5   -1.825306
dtype: float64

In [47]:
s.fillna(s.mean())

0   -0.993607
1   -1.190854
2   -0.993607
3    0.035340
4   -0.993607
5   -1.825306
dtype: float64

In [48]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East']*4 + ['West']*4
data = pd.Series(np.random.randn(8), index = states)
data

Ohio         -1.355364
New York     -0.962985
Vermont       0.612157
Florida      -0.496123
Oregon       -0.291138
Nevada        0.450634
California    0.041895
Idaho        -0.858416
dtype: float64

In [50]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio         -1.355364
New York     -0.962985
Vermont            NaN
Florida      -0.496123
Oregon       -0.291138
Nevada             NaN
California    0.041895
Idaho              NaN
dtype: float64

In [52]:
data.groupby(group_key).mean()

East   -0.938157
West   -0.124621
dtype: float64

순서대로 동쪽 서쪽 반영됨

In [53]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.355364
New York     -0.962985
Vermont      -0.938157
Florida      -0.496123
Oregon       -0.291138
Nevada       -0.124621
California    0.041895
Idaho        -0.124621
dtype: float64

In [54]:
fill_values = {'East':.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -1.355364
New York     -0.962985
Vermont       0.500000
Florida      -0.496123
Oregon       -0.291138
Nevada       -1.000000
California    0.041895
Idaho        -1.000000
dtype: float64

## 10.4 피벗테이블과 교차알림표

In [55]:
tips.pivot_table(index = ['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [56]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [58]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [59]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True, aggfunc=len)

size            tip_pct             
smoker          No   Yes  All      No   Yes    All
time   day                                        
Dinner Fri     3.0   9.0   12     3.0   9.0   12.0
       Sat    45.0  42.0   87    45.0  42.0   87.0
       Sun    57.0  19.0   76    57.0  19.0   76.0
       Thur    1.0   NaN    1     1.0   NaN    1.0
Lunch  Fri     1.0   6.0    7     1.0   6.0    7.0
       Thur   44.0  17.0   61    44.0  17.0   61.0
All          151.0  93.0  244   151.0  93.0  244.0

In [61]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True, aggfunc=len,fill_value=0)

size          tip_pct           
smoker        No Yes  All      No Yes    All
time   day                                  
Dinner Fri     3   9   12       3   9   12.0
       Sat    45  42   87      45  42   87.0
       Sun    57  19   76      57  19   76.0
       Thur    1   0    1       1   0    1.0
Lunch  Fri     1   6    7       1   6    7.0
       Thur   44  17   61      44  17   61.0
All          151  93  244     151  93  244.0

`pd.crosstab`을 사용해도 좋음